In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv("./data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("./data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("./data/8-1 test2(std).csv",index_col=0)


In [4]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test2.drop("label",axis=1)
y_test = test2["label"]

In [5]:
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(4646, 10)


label
0.0    2323
1.0    2323
Name: count, dtype: int64

In [6]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [7]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [8]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [9]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
오차행렬
[[1934  247]
 [   6   16]]
정확도: 0.8852, 정밀도 : 0.0608, 재현율:0.7273,F1 스코어:0.1123
ROC 스코어: 0.8070, PR score : 0.0470
임곗값: 0.1
오차행렬
[[1092 1089]
 [   3   19]]
정확도: 0.5043, 정밀도 : 0.0171, 재현율:0.8636,F1 스코어:0.0336
ROC 스코어: 0.6822, PR score : 0.0162
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1475  706]
 [   6   16]]
정확도: 0.6768, 정밀도 : 0.0222, 재현율:0.7273,F1 스코어:0.0430
ROC 스코어: 0.7018, PR score : 0.0188
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1681  500]
 [   6   16]]
정확도: 0.7703, 정밀도 : 0.0310, 재현율:0.7273,F1 스코어:0.0595
ROC 스코어: 0.7490, PR score : 0.0253
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1837  344]
 [   6   16]]
정확도: 0.8411, 정밀도 : 0.0444, 재현율:0.7273,F1 스코어:0.0838
ROC 스코어: 0.7848, PR score : 0.0350
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1934  247]
 [   6   16]]
정확도: 0.8852, 정밀도 : 0.0608, 재현율:0.7273,

In [10]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
임곗값: 0.1
오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2055  126]
 [  14    8]]
정확도: 0.9365, 정밀도 : 0.0597, 재현율:0.3636,F1 스코어:0.1026
ROC 스코어: 0.6529, PR score : 0.0281
---------

In [11]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 4}, 최적 평균 정확도 :0.921
디시전트리 파라미터적용
오차행렬
[[1950  231]
 [  14    8]]
정확도: 0.8888, 정밀도 : 0.0335, 재현율:0.3636,F1 스코어:0.0613
ROC 스코어: 0.6289, PR score : 0.0185
임곗값: 0.1
오차행렬
[[1855  326]
 [  11   11]]
정확도: 0.8470, 정밀도 : 0.0326, 재현율:0.5000,F1 스코어:0.0613
ROC 스코어: 0.6753, PR score : 0.0213
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1919  262]
 [  11   11]]
정확도: 0.8761, 정밀도 : 0.0403, 재현율:0.5000,F1 스코어:0.0746
ROC 스코어: 0.6899, PR score : 0.0251
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1926  255]
 [  11   11]]
정확도: 0.8793, 정밀도 : 0.0414, 재현율:0.5000,F1 스코어:0.0764
ROC 스코어: 0.6915, PR score : 0.0257
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1942  239]
 [  13    9]]
정확도: 0.8856, 정밀도 : 0.0363, 재현율:0.4091,F1 스코어:0.0667
ROC 스코어: 0.6498, PR score : 0.0207
---------------------------------------

In [12]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

디시전트리
오차행렬
[[1934  247]
 [   6   16]]
정확도: 0.8852, 정밀도 : 0.0608, 재현율:0.7273,F1 스코어:0.1123
ROC 스코어: 0.8070, PR score : 0.0470
None


In [13]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



랜덤포레스트
오차행렬
[[2145   36]
 [  17    5]]
정확도: 0.9759, 정밀도 : 0.1220, 재현율:0.2273,F1 스코어:0.1587
ROC 스코어: 0.6054, PR score : 0.0354
임곗값: 0.1
오차행렬
[[1698  483]
 [   5   17]]
정확도: 0.7785, 정밀도 : 0.0340, 재현율:0.7727,F1 스코어:0.0651
ROC 스코어: 0.7756, PR score : 0.0285
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1950  231]
 [   8   14]]
정확도: 0.8915, 정밀도 : 0.0571, 재현율:0.6364,F1 스코어:0.1049
ROC 스코어: 0.7652, PR score : 0.0400
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2066  115]
 [  10   12]]
정확도: 0.9433, 정밀도 : 0.0945, 재현율:0.5455,F1 스코어:0.1611
ROC 스코어: 0.7464, PR score : 0.0561
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2117   64]
 [  13    9]]
정확도: 0.9650, 정밀도 : 0.1233, 재현율:0.4091,F1 스코어:0.1895
ROC 스코어: 0.6899, PR score : 0.0563
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2145   36]
 [  17    5]]
정확도: 0.9759, 정밀도 : 0.1220, 재현율:0.2273,F1 스코어:0.1587
ROC 스코어: 0.6054, PR score : 0.0354
--

In [14]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}, 최적 평균 정확도 :0.965
랜덤포레스트 파라미터적용
오차행렬
[[2113   68]
 [  12   10]]
정확도: 0.9637, 정밀도 : 0.1282, 재현율:0.4545,F1 스코어:0.2000
ROC 스코어: 0.7117, PR score : 0.0637
임곗값: 0.1
오차행렬
[[1397  784]
 [   3   19]]
정확도: 0.6428, 정밀도 : 0.0237, 재현율:0.8636,F1 스코어:0.0461
ROC 스코어: 0.7521, PR score : 0.0218
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1760  421]
 [   5   17]]
정확도: 0.8066, 정밀도 : 0.0388, 재현율:0.7727,F1 스코어:0.0739
ROC 스코어: 0.7898, PR score : 0.0323
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1952  229]
 [   7   15]]
정확도: 0.8929, 정밀도 : 0.0615, 재현율:0.6818,F1 스코어:0.1128
ROC 스코어: 0.7884, PR score : 0.0451
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2049  132]
 [  10   12]]
정확도: 0.9355, 정밀도 : 0.0833, 재현율:0.5455,F1 스코어:0.1446
ROC 스코어: 0.7425, PR score : 0.0500
-------------------

In [15]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

로지스틱회귀
오차행렬
[[1916  265]
 [   6   16]]
정확도: 0.8770, 정밀도 : 0.0569, 재현율:0.7273,F1 스코어:0.1056
ROC 스코어: 0.8029, PR score : 0.0441
임곗값: 0.1
오차행렬
[[1038 1143]
 [   3   19]]
정확도: 0.4798, 정밀도 : 0.0164, 재현율:0.8636,F1 스코어:0.0321
ROC 스코어: 0.6698, PR score : 0.0155
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1426  755]
 [   4   18]]
정확도: 0.6555, 정밀도 : 0.0233, 재현율:0.8182,F1 스코어:0.0453
ROC 스코어: 0.7360, PR score : 0.0209
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1647  534]
 [   5   17]]
정확도: 0.7553, 정밀도 : 0.0309, 재현율:0.7727,F1 스코어:0.0593
ROC 스코어: 0.7639, PR score : 0.0261
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1804  377]
 [   6   16]]
정확도: 0.8261, 정밀도 : 0.0407, 재현율:0.7273,F1 스코어:0.0771
ROC 스코어: 0.7772, PR score : 0.0323
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1916  265]
 [   6   16]]
정확도: 0.8770, 정밀도 : 0.0569, 재현율:0.7273,F1 스코어:0.1056
ROC 스코어: 0.8029, PR score : 0.0441
--

In [16]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 5, 'penalty': 'l2'}, 최적 평균 정확도:0.827
로지스틱회귀 파라미터적용
오차행렬
[[1914  267]
 [   6   16]]
정확도: 0.8761, 정밀도 : 0.0565, 재현율:0.7273,F1 스코어:0.1049
ROC 스코어: 0.8024, PR score : 0.0438
임곗값: 0.1
오차행렬
[[1040 1141]
 [   3   19]]
정확도: 0.4807, 정밀도 : 0.0164, 재현율:0.8636,F1 스코어:0.0321
ROC 스코어: 0.6702, PR score : 0.0155
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1426  755]
 [   4   18]]
정확도: 0.6555, 정밀도 : 0.0233, 재현율:0.8182,F1 스코어:0.0453
ROC 스코어: 0.7360, PR score : 0.0209
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[1647  534]
 [   5   17]]
정확도: 0.7553, 정밀도 : 0.0309, 재현율:0.7727,F1 스코어:0.0593
ROC 스코어: 0.7639, PR score : 0.0261
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[1804  377]
 [   6   16]]
정확도: 0.8261, 정밀도 : 0.0407, 재현율:0.7273,F1 스코어:0.0771
ROC 스코어: 0.7772, PR score : 0.0323
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[1914  267]
 [   6   16]]
정확도: 0.8761, 정밀도 : 0.0565, 